# Note book to create html pages for countries and Kreise in Germany

In [1]:
%config InlineBackend.figure_formats = ['svg']
%cp -v ../coronavirus.py .
from coronavirus import *
import datetime

../coronavirus.py -> ./coronavirus.py


In [2]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [3]:
def modify_template(templatefile, output_file_name, mappings, wwwroot):
    """Create concrete *.ipynb file from template
    - templatefile: the template with placeholders to be substituted
    - mappings: dictiorany with placeholders as keys, and values to be substituted
    - output_file_name: name to write modified file to
    - wwwroot: directory in which the output file should be written
    """
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(os.path.join(wwwroot, output_file_name), "tw") as f:
        f.write(template)
    print(f"Written file to {output_file_name}")


In [4]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def germany_check_region_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Bundesland'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Bundesland'].drop_duplicates()))}"

def germany_check_subregion_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Landkreis'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Landkreis'].drop_duplicates()))}"

germany_check_region_name_is_known("Hamburg") 
germany_check_subregion_name_is_known("SK Hamburg") 

    
def sanitise(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    
    
def get_binder_url(notebook):
    base = "https://mybinder.org/v2/gh/fangohr/coronavirus/master?filepath="
    return base + notebook.replace(" ", "%20")


def create_ipynb_for_country(country, templatename, wwwroot):
    check_country_name_is_known(country)
    
    output_file_name =  f"{country}.ipynb"
    output_file_path = os.path.join(wwwroot, output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "",
        "%country%" : country,
        "%binderurl%" : get_binder_url(output_file_path),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M")
    }

    modify_template(templatename, output_file_name, mappings, wwwroot)
    assert os.path.exists(output_file_path)
    return output_file_name


In [5]:
import subprocess
def nbconvert_ipynb2html(ipynb_path, wwwroot):
    
    # copy file to run the notebook
    command = f"cp -fv ../coronavirus.py {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # copy requirements (needed for binder)
    command = f"cp -fv ../requirements.txt {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # execute notebook and create html copy from it
    command = f"""jupyter-nbconvert "{os.path.join(wwwroot, ipynb_path)}" """ + \
                " --to html --execute"
    print(f"Command = {command}")
    output = subprocess.check_call(command, shell=True)

    # compute output path
    output_file_name = os.path.splitext(ipynb_path)[0] + ".html"
    assert os.path.exists(os.path.join(wwwroot, output_file_name))
    
    return output_file_name
    

In [6]:
def create_md_index_list(title, links):
    lines = []
    lines.append(title)
    lines.append("")   # need empty line for markdown syntax
    for name, (path_html, path_ipynb) in links.items():
        lines.append(f"* [{name}]({path_html})")
    return "\n".join(lines)
        

In [7]:
def get_country_list():
    d, c = fetch_deaths(), fetch_cases()

    countries = d.index
    countries2 = c.index
    assert (countries2 == countries).all()
    
    # Here we should identify regions in countries, and process those.
    # Instead, as a quick hack to get started, we'll just take one country
    # and the current "get_country" method will sum over all regions of one country if only 
    # the country name is given.
    
    return sorted(countries.drop_duplicates())
    

In [12]:
def create_html_for_john_hopkins_countries(countries, wwwroot):
    start_time = time.time()
    if not os.path.exists(wwwroot):
        print("To put the html into github repo for webhosting, run")
        print('"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar')
        os.mkdir(wwwroot)
    

    created_files = {}
    country = "Italy"
 
    for i, country in enumerate(countries):
        print(f"Processing {i+1}/{len(countries)} [{time.time()-start_time:4.0f}s]")
        ipynb_path = create_ipynb_for_country(country, "template-country.ipynb", wwwroot=wwwroot)
        html_path = nbconvert_ipynb2html(os.path.join(ipynb_path), wwwroot=wwwroot)
        created_files[country] = html_path, ipynb_path
    print("Create {len(countries) notebooks and html versions in" + \
          "{time.time()-start_time} seconds}")
    return created_files

In [9]:
def create_index_page(sections, rootname, wwwroot):
    """Sections is dictionary: key is title, value is markdown text"""
    md_file = rootname + ".md"
    
    with open(os.path.join(wwwroot, md_file), "tw") as f:
        for section in sections:
            f.write(f"# {section}\n\n")
            f.write(sections[section])
    print(f"Written overview to {md_file}.")
    html_file = rootname + ".html"
    subprocess.check_call(f"pandoc -t html -o {os.path.join(wwwroot, html_file)} " +
                          f"{os.path.join(wwwroot, md_file)}", shell=True)
    return html_file

In [10]:
def get_germany_subregion_list():
    """returns list of subregions (Kreise), 
    ordered according to (i) Land, then (ii) Kreis
    """
    x = fetch_data_germany()
    land_kreis = x[['Bundesland', 'Landkreis']]
    ordered = land_kreis.sort_values(['Bundesland', 'Landkreis'])
    return list(ordered['Landkreis'].drop_duplicates())
 

@joblib_memory.cache
def germany_get_bundesland_from_kreis(kreis):
        x = fetch_data_germany()
        return x[x['Landkreis'] == kreis].iloc[0]['Bundesland']    

# Create country overview for the world

In [ ]:
wwwroot = "wwwroot"
countries = get_country_list()
created_files = create_html_for_john_hopkins_countries(countries, wwwroot)
index_md = create_md_index_list("Countries (data from Johns Hopkins University)", 
                                created_files)
intro ="[Additional plots for Germany](index-germany.html])\n\n"
create_index_page(sections={"Germany: Landkreise" : intro, 
                            "World" : index_md}, 
                  rootname="index", 
                  wwwroot=wwwroot)


Processing 1/184 [   0s]
Written file to Afghanistan.ipynb
Command = jupyter-nbconvert "wwwroot/Afghanistan.ipynb"  --to html --execute
Processing 2/184 [   6s]
Written file to Albania.ipynb
Command = jupyter-nbconvert "wwwroot/Albania.ipynb"  --to html --execute
Processing 3/184 [  12s]
Written file to Algeria.ipynb
Command = jupyter-nbconvert "wwwroot/Algeria.ipynb"  --to html --execute
Processing 4/184 [  18s]
Written file to Andorra.ipynb
Command = jupyter-nbconvert "wwwroot/Andorra.ipynb"  --to html --execute


In [11]:
print(create_md_index_list("Countries (data from Johns Hopkins University)", created_files))

Countries (data from Johns Hopkins University)

* [Afghanistan](Afghanistan.html)
* [Albania](Albania.html)
* [Algeria](Algeria.html)
* [Andorra](Andorra.html)
* [Angola](Angola.html)
* [Antigua and Barbuda](Antigua and Barbuda.html)
* [Argentina](Argentina.html)
* [Armenia](Armenia.html)
* [Australia](Australia.html)
* [Austria](Austria.html)
* [Azerbaijan](Azerbaijan.html)
* [Bahamas](Bahamas.html)
* [Bahrain](Bahrain.html)
* [Bangladesh](Bangladesh.html)
* [Barbados](Barbados.html)
* [Belarus](Belarus.html)
* [Belgium](Belgium.html)
* [Belize](Belize.html)
* [Benin](Benin.html)
* [Bhutan](Bhutan.html)
* [Bolivia](Bolivia.html)
* [Bosnia and Herzegovina](Bosnia and Herzegovina.html)
* [Botswana](Botswana.html)
* [Brazil](Brazil.html)
* [Brunei](Brunei.html)
* [Bulgaria](Bulgaria.html)
* [Burkina Faso](Burkina Faso.html)
* [Burma](Burma.html)
* [Burundi](Burundi.html)
* [Cabo Verde](Cabo Verde.html)
* [Cambodia](Cambodia.html)
* [Cameroon](Cameroon.html)
* [Canada](Canada.html)
* [Cen

# Create list of Germany data sets

In [43]:
@joblib_memory.cache
def create_html_for_Germany(subregions, wwwroot):
    start_time = time.time()
    created_files = {}

    if not os.path.exists(wwwroot):
        print("To put the html into github repo for webhosting, run")
        print('"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar')
        os.mkdir(wwwroot)

    for i, kreis in enumerate(subregions):
        bundesland = germany_get_bundesland_from_kreis(kreis)
        print(f"Processing {i+1}/{len(subregions)} [{time.time()-start_time:4.0f}s]")
        ipynb_path = create_ipynb_for_germany(region=bundesland, subregion=kreis, 
                                              templatename="template-germany.ipynb", wwwroot=wwwroot)
        html_path = nbconvert_ipynb2html(os.path.join(ipynb_path), wwwroot=wwwroot)
        one_line_summary = f"Germany: {bundesland} : {kreis}"
        created_files[one_line_summary] = html_path, ipynb_path
        print("Create {len(subregions) notebooks and html versions in" + \
              "{time.time()-start_time} seconds}")
    return created_files

In [44]:
def create_ipynb_for_germany(region, subregion, templatename, wwwroot):
    germany_check_region_name_is_known(region)
    germany_check_subregion_name_is_known(subregion)
    
    output_file_name =  f"Germany-{sanitise(region)}-{sanitise(subregion)}.ipynb"
    output_file_path = os.path.join(wwwroot, output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : f"Germany: {subregion} ({region})",
        "%title2%" : "",
        "%region%" : region,
        "%subregion%" : subregion,
        "%binderurl%" : get_binder_url(output_file_name),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M")
    }

    modify_template(templatename, output_file_name, mappings, wwwroot)
    assert os.path.exists(output_file_path)
    return output_file_name


In [ ]:
wwwroot = "wwwroot"
subregions = get_germany_subregion_list()
created_files = create_html_for_Germany(subregions, wwwroot)
index_md = create_md_index_list("Landkreise in Germany", 
                                created_files)
create_index_page(sections={"Links to static pages for each Landkreis" : index_md}, 
                  rootname="index-germany", 
                  wwwroot=wwwroot)
